In [1]:
import sqlalchemy
import pandas as pd
import server.model.connection as smc
import server.view.dataframes as smd

In [3]:
conn = smc.engine.connect()

In [4]:
sql = sqlalchemy.text(
    "with current AS (SELECT s.event_id as event_id, s.match, date from schedules sched, "
    "status s WHERE sched.event_id = s.event_id "
    "AND sched.match = s.match limit 1 ) "
    "SELECT * FROM current;")
pd.read_sql(sql, conn)

,event_id,match,date
0,25137,004-q,2018-02-17T09:27:00


In [5]:
num_matches = 12
sql = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc) "
    
    "SELECT * from recent_matches;"
)
pd.read_sql(sql, conn).head(20)

,r,id,date,level,match,alliance,team,station,event_id
0,1,54454,2018-02-17T09:09:00,qual,002-q,red,1153,2,25137
1,1,54468,2018-02-17T09:27:00,qual,004-q,blue,1247,1,25137
2,1,54467,2018-02-17T09:27:00,qual,004-q,red,125,3,25137
3,1,54465,2018-02-17T09:27:00,qual,004-q,red,1519,1,25137
4,1,54449,2018-02-17T09:00:00,qual,001-q,red,1721,3,25137
5,1,54450,2018-02-17T09:00:00,qual,001-q,blue,1729,1,25137
6,1,54463,2018-02-17T09:18:00,qual,003-q,blue,2342,2,25137
7,1,54459,2018-02-17T09:18:00,qual,003-q,red,238,1,25137
8,1,54451,2018-02-17T09:00:00,qual,001-q,blue,319,2,25137
9,1,54464,2018-02-17T09:18:00,qual,003-q,blue,3236,3,25137


In [6]:
select_sum = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc), "
    
    "team_match_count as ( "
    "select team, count(team) as team_matches from recent_matches group by team"
       ") "
       
    "SELECT * FROM team_match_count;"
)
pd.read_sql(select_sum, conn).head()

,team,team_matches
0,811,1
1,6153,1
2,3467,1
3,3236,1
4,1153,1


In [9]:
evt_id = 25137
sql = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc), "
    
    "team_match_count as ( "
    "select team, count(team) as team_matches from recent_matches group by team"
       ") "
    
    "SELECT teams.name AS team, phases.name AS phase, tasks.name AS task, actors.name AS actor, "
    "MAX(team_match_count.team_matches) AS matches, "
    "SUM(successes) AS sum_successes, MAX(successes) as max_successes, MIN(successes) as min_successes, "
    "COUNT(successes) AS count_successes, AVG(successes) as avg_successes, "
    "SUM(attempts) AS sum_attempts, MAX(attempts) as max_attempts, MIN(attempts) AS min_attempts, "
    "COUNT(attempts) AS count_attempts, AVG(attempts) as avg_attempts, "
    "MAX(cycle_times) AS max_cycle_times, MIN(cycle_times) AS min_cycle_times, "
    "AVG(cycle_times), COUNT(cycle_times) AS count_cycle_times "
    "FROM ((((teams FULL OUTER JOIN measures ON teams.id=measures.team_id) "
    "LEFT JOIN tasks ON tasks.id = measures.task_id) "
    "LEFT JOIN phases ON phases.id = measures.phase_id) "
    "LEFT JOIN events ON events.id = measures.event_id) "
    "LEFT JOIN actors ON actors.id = measures.actor_id "
    "LEFT JOIN matches ON matches.id = measures.match_id "
    "LEFT JOIN team_match_count ON team_match_count.team = teams.name "
    "RIGHT JOIN recent_matches ON recent_matches.match = matches.name AND team_match_count.team = teams.name "
    "AND recent_matches.team = team_match_count.team "
    "WHERE events.id = " + str(evt_id) + " AND actors.name<> 'alliance' "
    "GROUP BY teams.name, tasks.name, phases.name, actors.name "
    "ORDER BY teams.name, phases.name, tasks.name, actors.name;"
)
df = pd.read_sql(sql, conn)
df

,team,phase,task,actor,matches,sum_successes,max_successes,min_successes,count_successes,avg_successes,sum_attempts,max_attempts,min_attempts,count_attempts,avg_attempts,max_cycle_times,min_cycle_times,avg,count_cycle_times
0,1153,auto,autoLine,robot,1,1,1,1,1,1.0,1,1,1,1,1.0,0,0,0.0,1
1,1153,teleop,pickupPortal,robot,1,0,0,0,1,0.0,0,0,0,1,0.0,0,0,0.0,1
2,1153,teleop,placeSwitch,robot,1,1,1,1,1,1.0,1,1,1,1,1.0,0,0,0.0,1
3,1721,teleop,pickupFloor,robot,1,1,1,1,1,1.0,1,1,1,1,1.0,0,0,0.0,1
4,1721,teleop,placeSwitch,robot,1,2,2,2,1,2.0,2,2,2,1,2.0,0,0,0.0,1
5,1729,auto,autoLine,robot,1,0,0,0,1,0.0,0,0,0,1,0.0,0,0,0.0,1
6,1729,auto,crossNull,robot,1,0,0,0,1,0.0,0,0,0,1,0.0,0,0,0.0,1
7,1729,auto,placeIncorrect,robot,1,0,0,0,1,0.0,0,0,0,1,0.0,0,0,0.0,1
8,1729,auto,placeScale,robot,1,0,0,0,1,0.0,0,0,0,1,0.0,0,0,0.0,1
9,1729,auto,placeSwitch,robot,1,0,0,0,1,0.0,0,0,0,1,0.0,0,0,0.0,1


In [10]:
smd.ranking_df(12).head()

phase     auto                                                            \
actor    robot                                                             
task  autoLine                                                             
       matches sum_successes max_successes min_successes count_successes   
team                                                                       
1153       1.0           1.0           1.0           1.0             1.0   
1721       NaN           NaN           NaN           NaN             NaN   
1729       1.0           0.0           0.0           0.0             1.0   
2342       1.0           1.0           1.0           1.0             1.0   
238        NaN           NaN           NaN           NaN             NaN   

phase                                                                      \
actor                                                                       
task                                                                        
      avg_successes sum_attempts max_attempts min_attempts count_attempts   
team                                                                        
1153            1.0          1.0          1.0          1.0            1.0   
1721            NaN          NaN          NaN          NaN            NaN   
1729            0.0          0.0          0.0          0.0            1.0   
2342            1.0          1.0          1.0          1.0            1.0   
238             NaN          NaN          NaN          NaN            NaN   

phase        ...               teleop                                         \
actor        ...                robot                                          
task         ...            stackCube                                          
             ...        avg_successes sum_attempts max_attempts min_attempts   
team         ...                                                               
1153         ...                  NaN          NaN          NaN          NaN   
1721         ...                  NaN          NaN          NaN          NaN   
1729         ...                  0.0          0.0          0.0          0.0   
2342         ...                  NaN          NaN          NaN          NaN   
238          ...                  NaN          NaN          NaN          NaN   

phase                                                                   \
actor                                                                    
task                                                                     
      count_attempts avg_attempts max_cycle_times min_cycle_times  avg   
team                                                                     
1153             NaN          NaN             NaN             NaN  NaN   
1721             NaN          NaN             NaN             NaN  NaN   
1729             1.0          0.0             0.0             0.0  0.0   
2342             NaN          NaN             NaN             NaN  NaN   
238              NaN          NaN             NaN             NaN  NaN   

phase                    
actor                    
task                     
      count_cycle_times  
team                     
1153                NaN  
1721                NaN  
1729                1.0  
2342                NaN  
238                 NaN  

[5 rows x 345 columns]